In [1]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline

In [2]:
# from transformers import pipeline
# generator = pipeline('text-generation', model = 'EleutherAI/polyglot-ko-5.8b', device="cuda")
# generator("안녕하세요, ", max_length = 30, num_return_sequences=1)
# ## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},
# ##  {'generated_text': "Hello, I'm a language modeler. I write and maintain software in Python. I love to code, and that includes coding things that require writing"}, ...


Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': '안녕하세요, ^^*오늘은 제가 좋아하는 맛집을 소개해드리려고 해요.바로바로~~~~~~'}]

In [ ]:
# generator("사람에게 물이 부족하면", max_length = 120, num_return_sequences=1)

In [1]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
from datasets import load_from_disk
import evaluate

import numpy as np

In [2]:
# str_model = 'KETI-AIR/ke-t5-base'
str_model ="paust/pko-t5-base"

model = AutoModelForSeq2SeqLM.from_pretrained(str_model)
tokenizer = AutoTokenizer.from_pretrained(str_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
path_dataset = "./dataset/paper/paper_dataset"

dataset_paper = load_from_disk(path_dataset)
dataset_paper

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'ipc', 'all', 'sum'],
        num_rows: 287982
    })
    test: Dataset({
        features: ['doc_id', 'title', 'ipc', 'all', 'sum'],
        num_rows: 578
    })
})

In [4]:
prefix = "요약: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["all"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["sum"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [5]:
dataset_paper_tokenized = dataset_paper.map(preprocess_function, batched=True)

Loading cached processed dataset at /opt/ml/dataset/paper/paper_dataset/train/cache-379987f59dadf2bb.arrow


Map:   0%|          | 0/578 [00:00<?, ? examples/s]

In [106]:
print("전문")
print(dataset_paper_tokenized["train"][100]["all"])
print("요약")
print(dataset_paper_tokenized["train"][100]["sum"])
print()
print(tokenizer.decode(dataset_paper_tokenized["train"][100]["labels"]))

전문
2012년 경상남도 진주시 농산물도매시장에 판매중인 감귤에서 분홍빛열매썩음병이 발생하였다. 병징은 감귤 과실 표면이 수침상으로 물러지고 썩으면서 그 위에 분홍빛 곰팡이가 많이 형성되었다. 균총의 색깔은 처음에 흰색이고 배양기간이 경과됨에 따라 배지 표면에 분홍빛의분생포자가 많이 형성되었다. 균사생육 적온은 25oC이었다. 분생포자의 모양은 서양배형이며 좌우 zigzag로 부착하며 성숙한 분생포자는 2세포로 되어 있으며 크기는 12?26 × 8?12 μm이었다. 분생자경은 균사표면에 직립으로 형성하고, 폭이 4-5 μm이고 무색이었다. rDNA의 completeinternal transcribed spacer(ITS) 영역의 염기서열을 분석하였고, 분석된 염기서열(613 bp)을 BLASTN 프로그램으로확인한 결과, Trichothecium roseum와 99%의 상동성을 나타내었다. 이와 같이 감귤에서 발생한 병징과 병원균의균학적 특징을 기초로 하여 이 병을 Trichothecium roseum(Pers.) Link ex Gray에 의한 감귤 분홍빛열매썩음병으로명명하고자 제안한다.
요약
2012년 경상남도 진주시 농산물도매시장에 판매중인 감귤에서 분홍빛열매썩음병이 발생했다. 분생포자의 모양은 서양배형이며 좌우 zigzag로 부착하며 성숙한 분생포자는 2세포로 되어 있으며 크기는 12?26 × 8?12 μm이었다.

2012년 경상남도 진주시 농산물도매시장에 판매중인 감귤에서 분홍빛열매썩음병이 발생했다. 분생포자의 모양은 서양배형이며 좌우 zigzag로 부착하며 성숙한 분생포자는 2세포로 되어 있으며 크기는 12?26 × 8?12 μm이었다.</s>


In [7]:
rouge = evaluate.load("rouge")

In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(predictions, labels)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=str_model)

training_args = Seq2SeqTrainingArguments(
    output_dir="summary",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # gradient_accumulation_steps=2,
    # eval_accumulation_steps=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    eval_steps=2000,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_paper_tokenized["train"],
    eval_dataset=dataset_paper_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
model(dataset_paper_tokenized["train"][0]["input_ids"])

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
2000,0.321900,0.312891,0.097100,0.026200,0.097500,0.097700,18.994800
4000,0.279300,0.293201,0.096800,0.025400,0.097000,0.097200,19.000000
6000,0.271500,0.279073,0.099900,0.028000,0.100100,0.100300,19.000000
8000,0.269900,0.271338,0.098600,0.027200,0.098800,0.099300,19.000000
10000,0.271100,0.262079,0.100300,0.025400,0.100700,0.100900,19.000000
12000,0.262000,0.264940,0.100400,0.026200,0.100700,0.100600,19.000000
14000,0.266400,0.255469,0.102700,0.026400,0.102700,0.103100,19.000000
16000,0.257400,0.248780,0.099900,0.025400,0.100100,0.100100,19.000000
18000,0.241500,0.255119,0.100100,0.025800,0.100500,0.101000,19.000000
20000,0.242700,0.242868,0.097600,0.026400,0.098000,0.098300,19.000000


[[   0  262  222 ... 6596  476  222]
 [   0  262  222 ... 1838  389  222]
 [   0 7717  222 ... 1018 1024  222]
 ...
 [   0 2739  222 ...  222 1440  333]
 [   0 1373  222 ...  222 8589  222]
 [   0 4376  222 ...  248 8497  302]] [[ 262  222 1271 ...  681  222    1]
 [ 262  222 1271 ...  222  517    1]
 [7717  222 1164 ... -100 -100 -100]
 ...
 [ 262  222 1271 ... -100 -100 -100]
 [ 262  222 6851 ... -100 -100 -100]
 [4376  222 8796 ...  222  657    1]]
[[    0   262   222 ...  6596   476   222]
 [    0   262   222 ...  1838   389   222]
 [    0  7717   222 ...  1018  1024   222]
 ...
 [    0   262   222 ...  1001  2452   279]
 [    0 43327   972 ...  1085   222  1630]
 [    0  4376   222 ...   248  8497   302]] [[ 262  222 1271 ...  681  222    1]
 [ 262  222 1271 ...  222  517    1]
 [7717  222 1164 ... -100 -100 -100]
 ...
 [ 262  222 1271 ... -100 -100 -100]
 [ 262  222 6851 ... -100 -100 -100]
 [4376  222 8796 ...  222  657    1]]
[[    0   262   222 ...  6596   476   222]
 [    0  

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[[    0   262   222 ...  6596   476   222]
 [    0   582   222 ...  1838   389   222]
 [    0  7717   222 ...  1018  1024   222]
 ...
 [    0   262   222 ...  1001  2452   279]
 [    0 43327   972 ...  1085   222  1630]
 [    0  4376   222 ...   248  8497   302]] [[ 262  222 1271 ...  681  222    1]
 [ 262  222 1271 ...  222  517    1]
 [7717  222 1164 ... -100 -100 -100]
 ...
 [ 262  222 1271 ... -100 -100 -100]
 [ 262  222 6851 ... -100 -100 -100]
 [4376  222 8796 ...  222  657    1]]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[[    0   262   222 ...  6596   476   222]
 [    0   582   222 ...  1838   389   222]
 [    0  7717   222 ...  1018  1024   222]
 ...
 [    0   262   222 ...  1001  2452   279]
 [    0 43327   972 ...  1085   222  1630]
 [    0  4376   222 ...   248  8497   302]] [[ 262  222 1271 ...  681  222    1]
 [ 262  222 1271 ...  222  517    1]
 [7717  222 1164 ... -100 -100 -100]
 ...
 [ 262  222 1271 ... -100 -100 -100]
 [ 262  222 6851 ... -100 -100 -100]
 [4376  222 8796 ...  222  657    1]]
[[    0   262   222 ...  6596   476   222]
 [    0   582   222 ...  1838   389   222]
 [    0  7717   222 ...  1018  1024   222]
 ...
 [    0   262   222 ...  1001  2452   279]
 [    0 43327   972 ...  1085   222  1630]
 [    0  4376   222 ...   248  8497   302]] [[ 262  222 1271 ...  681  222    1]
 [ 262  222 1271 ...  222  517    1]
 [7717  222 1164 ... -100 -100 -100]
 ...
 [ 262  222 1271 ... -100 -100 -100]
 [ 262  222 6851 ... -100 -100 -100]
 [4376  222 8796 ...  222  657    1]]


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[[    0   262   222 ...  6596   476   222]
 [    0   582   222 ...  1838   389   222]
 [    0  7717   222 ...  1018  1024   222]
 ...
 [    0   262   222 ...  1001  2452   279]
 [    0 43327   972 ...  1085   222  1630]
 [    0  4376   222 ...   248  8497   302]] [[ 262  222 1271 ...  681  222    1]
 [ 262  222 1271 ...  222  517    1]
 [7717  222 1164 ... -100 -100 -100]
 ...
 [ 262  222 1271 ... -100 -100 -100]
 [ 262  222 6851 ... -100 -100 -100]
 [4376  222 8796 ...  222  657    1]]


KeyboardInterrupt: 

In [ ]:
model(tokenizer("우아아아아아", max_length=1024, truncation=True, return_tensors="pt")["input_ids"].to("cuda"))

In [ ]:
import accelerate
accelerate.__version__

In [ ]:
# export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [69]:
from transformers import pipeline

summarizer = pipeline("summarization", model="./summary/checkpoint-128000")
# summarizer = pipeline("text2text-generation", model="./summary/checkpoint-128000")
summarizer("요약: " + "GPT-3를 필두로 하여 거대 파라미터의 모델에 대규모 코퍼스를 학습한 (초)거대 언어모델은 자연스러운 문장을 생성하거나 다양한 태스크를 적은 학습 데이터로 수행하는 등 뛰어난 가능성을 보였다. 하지만 학습에 막대한 자본이 필요한 거대 언어모델은 AI의 독점화 및 권력화, 그리고 데이터 및 컴퓨팅 파워 격차에 따른 기술 격차 심화를 낳을 것이라는 우려도 존재한다. 빅테크 기업이 아닌 일반 기업이나 연구자가 거대 언어 모델을 다루기는 쉽지 않은 것은 엄연한 사실이다.")


[{'summary_text': '빅테크 기업이 아닌 일반 기업이나 연구자가 거대 언어'}]

In [110]:
full_news = """
뚜렷한 이유는 안밝혀..지난해 사업협력 단순투자로 공시했는데네이버 카페 주주모임, 불매운동하자는 글까지3명 사외이사 재선임 최종 부결돼도상법상 임시주총까지 퇴임이사로 활동 가능[이데일리 김현아 기자]KT의 지분 7.79%(현대자동차 4.69%, 현대모비스 3.1%)를 보유한 현대자동차그룹이 오는 31일 KT 주주총회에서 KT 사외이사 3명의 재선임안(임기 1년)에 대해 반대하겠다는 뜻을 KT에 전달한 것으로 확인되자, KT 개인주주들이 분노하고 있다. 현대차그룹은 강충구 고려대 전기전자공학부 교수, 여은정 중앙대 경영학부 교수, 표현명  한국타이어테크놀로지 사외이사 등 3명에 대해 반대의 뜻을 KT에 전했다. 앞서 현대차그룹은 윤경림 차기 CEO 후보에 대해서도 “이사회가 대주주 의견을 고려해야 한다”는 취지를 밝혀, 사실상 최대 주주인 국민연금과 뜻을 함께 하겠다는 의지를 밝힌 바 있다. 지난해 KT와 현대차는 7500억 규모의 자사주 맞교환을 통해 도심항공모빌리티(UAM) 등 미래 모빌리티 분야의 경쟁력을 키우기로 합의한 바 있다. 현대차 그룹이 KT 지분 4.6%, 현대모비스가 KT 지분 3.1%를, KT가 현대차 지분 1.04%와 현대모비스 지분 1.46%를 갖게 됐다.
"""

def get_summary(full_news):
    news_tokenized = len(tokenizer(full_news)["input_ids"])
    print(news_tokenized)
    
    return summarizer(full_news, min_length=news_tokenized // 2, max_length=news_tokenized // 2 + (news_tokenized // 4))

# 1934

# [{'generated_text': '제주항공에 자금을 지원하기 위해 제주항공 주식을 담보로 돈을 빌리는 AK홀딩스, SK케미칼 주식 공개매수 나서는 SK디스커버리, 하나금융14호스팩으로 본 스팩의 상장폐지 절차와 관련한 이야기 등을 담아봤어요.AK홀딩스 제주항공 유증 참여 위해 교환사채 발행애경그룹 지주회사AK홀딩스가 자회사인 저비용항공사제주항공의 주주배정 유상증자 참여를 위해 1300억원 )                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 '}]


In [112]:
list_news = ["뚜렷한 이유는 안밝혀..지난해 사업협력 단순투자로 공시했는데네이버 카페 주주모임, 불매운동하자는 글까지3명 사외이사 재선임 최종 부결돼도상법상 임시주총까지 퇴임이사로 활동 가능[이데일리 김현아 기자]KT의 지분 7.79%(현대자동차 4.69%, 현대모비스 3.1%)를 보유한 현대자동차그룹이 오는 31일 KT 주주총회에서 KT 사외이사 3명의 재선임안(임기 1년)에 대해 반대하겠다는 뜻을 KT에 전달한 것으로 확인되자, KT 개인주주들이 분노하고 있다. 현대차그룹은 강충구 고려대 전기전자공학부 교수, 여은정 중앙대 경영학부 교수, 표현명  한국타이어테크놀로지 사외이사 등 3명에 대해 반대의 뜻을 KT에 전했다. 앞서 현대차그룹은 윤경림 차기 CEO 후보에 대해서도 “이사회가 대주주 의견을 고려해야 한다”는 취지를 밝혀, 사실상 최대 주주인 국민연금과 뜻을 함께 하겠다는 의지를 밝힌 바 있다. 지난해 KT와 현대차는 7500억 규모의 자사주 맞교환을 통해 도심항공모빌리티(UAM) 등 미래 모빌리티 분야의 경쟁력을 키우기로 합의한 바 있다. 현대차 그룹이 KT 지분 4.6%, 현대모비스가 KT 지분 3.1%를, KT가 현대차 지분 1.04%와 현대모비스 지분 1.46%를 갖게 됐다.",
"당시 양사는 “사업협력을 위한 단순투자”라면서 지분투자 목적을 ‘단순투자’로 공시했다. 하지만, 이 같은 보도가 나오자 개인주주들은 반발하고 있다. 네이버 KT주주모임 카페에서 아이디 알바트로스님은 “현대차가 선을 넘는건가요?”라고 했고, chsu6366님은 “현대차그룹의 양아치 경영에 분노를 표하며 주주들이 불매운동에 적극적으로 동참하시길 당부드린다. KT CFO는 현대차그룹 지분을 매각하시길 바란다”고 했다. 앞서 주주모임 카페 개설자도 이데일리와의 통화에서 “이번 주총에서 현대차, 신한은행이 반대하면, 주주제안으로 미래사업 제휴에서 두 회사를 제외하고 KT와 상호주식교환 등을 한 걸 해지하라고 요구할 예정이다.",
"카페 안에선 굉장히 좋은 현대차에 대해 불매 운동까지 하자는 글도 있다”고 밝힌 바 있다. 한편 이강철 이사에 이어 어제(28일),김대유 이사(DB생명 사외이사)와 유희열 이사(한국 이산화탄소 포집 및 처리 연구개발센터(KCRC) 이사장) 등 지난 정부 출신 사외이사들이 사퇴하면서 KT 이사회는 사내이사 0명, 사외이사 4명이 남은 상태다. 이중 강충구, 여은정, 표현명 이사가 주총에서 재선임되지 않으면 이사는 1명만 남게 된다. 현대차 반대해 최종 부결돼도 상법으로 이사 의무 유지하지만, 법조계에 따르면 설사 KT 이사가 한 명도 안 남아도 상법상 이사의 결원 조항으로 인해 새 이사회 구성까지 임무를 담당할 수 있다. 퇴임이사 자격으로 가능하다.",
"ESG 자문기관인 서스틴베스트 자문 변호사에 따르면, 상법 386조 1항에 따라, 법률 또는 정관에서 정한 이사의 원수를 결한 경우에는 임기의 만료 또는 사임으로 인하여 퇴임한 이사는 새로 선임된 이사가 취임할 때까지 이사로서의 권리와 의무가 있다고 돼 있다. KT 이사회의 최소 의결 정족 원수는 3명인데, 현대차 주장대로 선임안이 부결되면 이사는 1명 남게 된다. 하지만, 부결돼도 해당 이사들은 이 조항에 근거해 임시주총에서 새로운 이사가 추천돼 선임될 때까지 퇴임이사로 활동할 수 있다. 한편 KT의 주가는 다시 3만 원대 아래로 추락했다. 29일 현재 2만 9200원이 돼 전날보다 2.83% 하락한 채 마감했다."]

for news in list_news:
    print(get_summary(news))

436
[{'summary_text': 'KT의 지분 7.79%를 보유한 현대자동차그룹이 오는 31일 KT 주주총회에서 KT 사외이사 3명의 재선임안에 대해 반대하겠다는 뜻을 KT에 전달한 것으로 확인되자, KT 개인주주들이 분노하고 있다. 현대차그룹은 강충구 고려대 전기전자공학부 교수, 여은정 중앙대 경영학부 교수, 표현명 한국타이어테크놀로지 사외이사 등 3명에 대해                                                                                        '}]
254
[{'summary_text': '네이버 KT주주모임 카페에서 아이디 알바트로스님은 “현대차가 선을 넘는건가요?”라고 했고, chsu6366님은 “현대차그룹의 양아치 경영에 분노를 표하며 주주들이 불매운동에 적극적으로 동참하시길 당부드린다. KT CFO는 현대차그룹 지분을 매각하시길 바란다”고 했다. 앞서 주주모임 카페 개설자도 이데일리와의 통화에서 �'}]
265
[{'summary_text': '강충구, 여은정, 표현명 이사가 주총에서 재선임되지 않으면 이사는 1명만 남게 된다. 현대차 반대해 최종 부결돼도 상법으로 이사 의무 유지하지만, 법조계에 따르면 설사 KT 이사가 한 명도 안 남아도 상법상 이사의 결원 조항으로 인해 새 이사회 구성까지 임무를 담당할 수 있다. 퇴임이사 자격으로 가능하다.\n\n퇴임이사 자격으로 '}]
252
[{'summary_text': 'KT 이사회의 최소 의결 정족 원수는 3명인데, 현대차 주장대로 선임안이 부결되면 이사는 1명 남게 된다. 하지만, 부결돼도 해당 이사들은 이 조항에 근거해 임시주총에서 새로운 이사가 추천돼 선임될 때까지 퇴임이사로 활동할 수 있다. 한편 KT의 주가는 다시 3만 원대 아래로 추락했다. 29일 현재 2만 9200원이 '}]
